In [796]:
from selenium import webdriver
import time
import pandas as pd
import os

In [797]:
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [798]:
#Use this Url and change city or role accordingly

url1="https://www.linkedin.com/jobs/search?keywords=software%20&location=uzbekistan&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"

In [799]:
#Load the web driver and get the url

driver = webdriver.Chrome(executable_path='C:/Users/Muhammadyusuf/chromedriver.exe')
driver.implicitly_wait(10)
driver.get(url1)

C:\Users\Muhammadyusuf\AppData\Local\Temp\ipykernel_748\1517794196.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/Muhammadyusuf/chromedriver.exe')


In [800]:
# #Loop to scroll through all jobs and click on see more jobs button for infinite scrolling

# i = 2
# while i <= int((n+200)/25)+1: 
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     i = i + 1
    
    
#     try:
#         send=driver.find_element_by_xpath("//button[@aria-label='Load more results']")
#         driver.execute_script("arguments[0].click();", send)   
#         time.sleep(3)
    
        
#         buu=driver.find_elements_by_tag_name("button")
#         x=[btn for btn in buu if btn.text=="See all jobs"]
#         for btn in x:
#             driver.execute_script("arguments[0].click();", btn)
#             time.sleep(3)
        
                                                 


            
#     except:
#         pass
#         time.sleep(5)
         

In [801]:
#Find number of job listings
y = driver.find_elements(By.CLASS_NAME, 'results-context-header__job-count')[0].text

In [802]:
type(y)

str

In [803]:
y

'116'

In [804]:
value_str = y.replace(",", "").replace("+", "")
n = int(value_str)

In [805]:
n

116

In [806]:
#Create empty lists for company name and job title
companyname= []
titlename= []

In [807]:
#Find company name and append it to the blank list
elements = driver.find_elements(By.CLASS_NAME, 'base-search-card__subtitle')
for element in elements:
    companyname.append(element.text)

In [808]:
#Find title name and append it to the blank list
elements = driver.find_elements(By.CLASS_NAME, 'base-search-card__title')
for element in elements:
    titlename.append(element.text)

In [809]:
from selenium.common.exceptions import TimeoutException
import time
import re

requirements_list=[]
years=[]
locations=[]
description=[]

for element in elements:
    try:
        # Click on the job title
        actions = ActionChains(driver)
        actions.move_to_element(element).click().perform()

        # Wait for job description to load
        time.sleep(2)
        
        from bs4 import BeautifulSoup

        # Get the page source
        page_source = driver.page_source

        # Create a BeautifulSoup object
        soup = BeautifulSoup(page_source, 'html.parser')

        # Find the requirements section
        sections = ['Requirements:', 'Required Skills:', 'Responsibilities:', 'What You’ll Need', 'What You Have', 'Qualifications:', 'Experience:', 'Essential Skills:', 'Desired Skills:', 'Education:', 'Certifications:', 'Technical Skills:', 'Soft Skills:', 'Abilities:']

        requirements_section = None
        for synonym in synonyms:
            requirements_section = soup.find('strong', text=synonym)
            if requirements_section:
                requirements_section = requirements_section.find_next('ul')
                break

        # Extract the requirements text
        if requirements_section:
            requirements = '\n'.join([li.text for li in requirements_section.find_all('li')])
            #print(requirements)
            requirements_list.append(requirements)
            
            pattern1 = r"(\d+)[\s-]*((?:to)?[\s-]*\d+)?[\s-]*(?:years?|yrs?)"
            pattern2 = r'\b\d+[\s+-]*(?:to)?[\s+-]*\d*\b|\b\ds*\+\b|\b(?:more than|over)\s*\d+\b'

            match1 = re.search(pattern1, requirements)
            match2 = re.search(pattern2, requirements)

            if match1 or match2:
                years_of_experience = match1.group(0) if match1 else match2.group(0)
                #print(years_of_experience)
                years.append(years_of_experience)
            else:
                #print("No years of experience found.")
                years.append("Not mentioned.")
        else:
            #print('No requirements found.')
            requirements_list.append("Could not find requirements.")
            years.append("Not mentioned.")
        
        
        location_element = soup.find('span', {'class': 'topcard__flavor topcard__flavor--bullet'})
        location = location_element.get_text(strip=True)
        locations.append(location) 
       



        # Switch back to the main content
        driver.switch_to.default_content()



        # Scroll down to the next job title
        actions = ActionChains(driver)
        actions.move_to_element(element).send_keys(Keys.DOWN).perform()

        # Wait for the next job title to load before proceeding
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'base-search-card__title')))

    except (TimeoutException, NoSuchElementException) as e:
        print(f"Encountered error: {e}")
        # Continue to the next job title if there is an error
        continue


In [810]:
data = {
    'Company Name': companyname,
    'Job Name': titlename,
    'Job Location':locations,
    'Years of Experience Required': years,
    'Requirements': requirements_list
}
df = pd.DataFrame(data, index=range(1, len(companyname) + 1))

In [811]:
len(df)

25

In [812]:
df['Years of Experience Required'].value_counts()

Not mentioned.    12
1+                12
5+                 1
Name: Years of Experience Required, dtype: int64

In [813]:
df

,Company Name,Job Name,Job Location,Years of Experience Required,Requirements
1,ICAS Technology (S) Pte Ltd,Software Developer,"Surxondaryo Region, Uzbekistan",Not mentioned.,Could not find requirements.
2,AppCake,HTML5 Developer,"Surxondaryo Region, Uzbekistan",Not mentioned.,Could not find requirements.
3,AppCake,Backend Developer,"Surxondaryo Region, Uzbekistan",Not mentioned.,Could not find requirements.
4,DSR Corporation,Front-End Software Engineer (UZ),"Surxondaryo Region, Uzbekistan",Not mentioned.,Could not find requirements.
5,AppCake,Back-end Developer,"Surxondaryo Region, Uzbekistan",Not mentioned.,Could not find requirements.
6,AppCake,HTML Developer with Experience in Playable Ads,"Surxondaryo Region, Uzbekistan",Not mentioned.,Could not find requirements.
7,DSR Corporation,Java Windows Software Engineer (UZ),"Tashkent, Tashkent, Uzbekistan",1+,"1+ year of professional, relevant experience\n..."
8,DSR Corporation,Embedded Systems Software Engineer (UZ),"Tashkent, Tashkent, Uzbekistan",1+,"1+ year of professional, relevant experience\n..."
9,Tashkent University of Information Technologie...,Full stack developer,"Tashkent, Tashkent, Uzbekistan",1+,"1+ year of professional, relevant experience\n..."
10,DSR Corporation,C/C++ Linux Software Engineer (UZ),"Tashkent, Tashkent, Uzbekistan",1+,"1+ year of professional, relevant experience\n..."


In [814]:
#Save file in your directory
x.to_csv('linkedin_jobs.csv')

In [815]:
#IF NEEDED YOU CAN ADD THE LINK AS WELL.
# #Find job links and append it to a list

# jobList = driver.find_elements_by_class_name('base-card__full-link')
# hrefList = []
# for e in jobList:
#     hrefList.append(e.get_attribute('href'))

# #for href in hrefList:
#     #link.append(href)
    

In [816]:
#Close the driver
driver.close()